In [1]:
dependencies <- c('readxl','ggplot2','dplyr','sp','rgdal','tmap','plyr','dplyr','forcats','lubridate','geojsonio','ggfortify','tseries','forecast')
lapply(dependencies, library, character.only = TRUE)



Attaching package: ‘dplyr’



The following objects are masked from ‘package:stats’:

    filter, lag



The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



rgdal: version: 1.3-3, (SVN revision 759)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-7 


In [0]:
ops_recent_raw <- read.csv('op_recent.csv', stringsAsFactors = FALSE)

In [0]:
wards <- geojson_read("ward_locations.geojson", what = 'sp')

In [0]:
weather <- read_excel("weather.xlsx", col_types = c("date", 
                                                    "blank", "blank", "blank", "blank", "blank", 
                                                    "blank", "blank", "blank", "numeric", 
                                                    "blank", "blank", "blank", "blank", "blank", 
                                                    "numeric", "blank", "blank", "blank", 
                                                    "blank", "blank", "blank", "blank", "blank", 
                                                    "blank", "blank", "blank"))

In [0]:
weather$didRain <- weather$`Total Rain (mm)` != 0.0
weather$`Total Rain (mm)` <- NULL
weather <- rename(weather, c("Mean Temp (°C)"="mean_tmp", "Date/Time" = "date"))
weather$date <- date(weather$date)
head(weather)

In [0]:
ops_recent <- data.frame(
  "date_time" = mdy_hms(op_recent_raw$incident_datetime),
  "date" = date(mdy_hms(op_recent_raw$incident_datetime)),
  "incident_type" = as.factor(op_recent_raw$incident_type_primary),
  "parent_type" = as.factor(op_recent_raw$parent_incident_type),
  "day" = as.factor(op_recent_raw$day_of_week),
  "hour" = op_recent_raw$hour_of_day,
  "long" = op_recent_raw$longitude,
  "lat" = op_recent_raw$latitude)

In [0]:
ops_recent <- left_join(ops_recent, weather)

In [0]:
plot(wards)

In [0]:
ops_recent$ward <-over(
  SpatialPointsDataFrame(ops_recent[,c("long","lat")], 
                         ops_recent, 
                         proj4string = CRS("+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0"))
  , wards)[,"WARD_EN"]
ops_recent <- ops_recent[complete.cases(ops_recent),]

In [0]:
wards_2 <- wards
wards_2@data <- left_join(wards_2@data, count(ops_recent, 'ward'), by=c("WARD_EN" = "ward"))
tm_shape(wards_2) + tm_fill("freq", title = "Number of Incidents") + tm_borders() + tm_layout("Crimes in Ottawa since 2018/04/22")